v9 latent_dim = 20 -> 200, dropout_rate = 0.4 -> 0.3
****
v12 n_epochs = 20 -> 225
***
v15 remove layers, image_size 128 -> 64, trainable_params 6e+6 -> 2e+6, n_epochs = 200 -> 50
***
v17 add batch normalization
***
v18 change data load method because ImageDataGenerator is suspected of malfunctioning
***
change vae model

## Import modules

In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

## Configuration class

In [ ]:
class Config:
    seed = 44
    n_epochs = 50
    batch_size = 32
    dropout_rate = 0.3
    latent_space_dim = 200
        
    validation_rate = 0.2
    
    image_size = 128
    channels = 3
    
    
    optimizer = 'adam'
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  mode='auto',
                                  factor=0.8,
                                  patience=2,
                                  epsilon=1e-4,
                                  coldown=5,
                                  min_lr=1e-5,
                                  verbose=1)
    
    checkpoint_best = ModelCheckpoint('best_model.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      verbose=1,
                                      save_best_only=True,
                                      save_weights_only=False)
    
    checkpoint_last = ModelCheckpoint('last_model.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      verbose=1,
                                      save_best_only=False,
                                      save_weights_only=False)
  
    
    callbacks = [reduce_lr, checkpoint_best, checkpoint_last]
    
    paths = {'images_folder': '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/',
             'list_attributes_file': '/kaggle/input/celeba-dataset/list_attr_celeba.csv'}

In [ ]:
config = Config()

## Data load

In [ ]:
def build_decoder(test=False, out_size=(config.image_size, config.image_size)):
    def decoder(path):
        img = file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels=config.channels)  
        img = tf.image.resize(img, (config.image_size, config.image_size))
        img = tf.cast(img, tf.float32) / 255.0
        return img
    def decoder_train(path):
        return decoder(path), decoder(path)

    return decoder if test else decoder_train

In [ ]:
def build_dataset(paths, test=False, shuffle=1, batch_size=1):
    AUTO = tf.data.experimental.AUTOTUNE
    decoder = build_decoder(test)

    dset = tf.data.Dataset.from_tensor_slices(paths)
    dset = dset.map(decoder, num_parallel_calls=AUTO)
    
    dset = dset.shuffle(shuffle)
    dset = dset.batch(batch_size)
    return dset

In [ ]:
image_paths = glob.glob(config.paths['images_folder'] + '*.jpg')

train_paths, valid_paths, _, _ = train_test_split(image_paths, image_paths, test_size=config.validation_rate, shuffle=True)

train_dataset = build_dataset(train_paths, batch_size=config.batch_size)
valid_dataset = build_dataset(valid_paths, batch_size=config.batch_size)

In [ ]:
plt.figure(figsize=(4,4))
for n, (image, label) in enumerate(train_dataset.unbatch().take(1)):
    f, (ax1, ax2) = plt.subplots(1, 2) 
    
    ax1.imshow(image)
    ax2.imshow(label)
plt.show()


## Model architecture

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Lambda, UpSampling2D, Conv2DTranspose, Reshape

from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    
    epsilon = K.random_normal(shape=(batch, config.latent_space_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def VAE(input_shape=(config.image_size, config.image_size, config.channels)):
    
    #Encoder
    input_encoder = Input(shape=(input_shape))
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(input_encoder)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)    
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)   
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)   
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)    
    x = Flatten()(x)

    latent_mu = Dense(config.latent_space_dim, name='latent_mean')(x)
    latent_log_var = Dense(config.latent_space_dim, name='latent_log_var')(x)
    latent_sample = Lambda(sampling)([latent_mu, latent_log_var])

    encoder = Model(input_encoder, [latent_mu, latent_log_var, latent_sample], name='encoder')

    latent_input = Input(shape=(config.latent_space_dim,), name='decoder_input')
    x = Dense(8 * 8 * config.image_size)(latent_input)
    x = Reshape((8, 8, config.image_size))(x)
    x = UpSampling2D((2, 2))(x)
    X = Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(X)
    X = Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    X = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    X = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    output_decoder = Conv2DTranspose(config.channels, (3, 3), activation='sigmoid', padding='same')(x)
    
    decoder = Model(latent_input, output_decoder, name='decoder')

    output_vae = decoder(encoder(input_encoder)[2])
    vae = Model(input_encoder, output_vae, name ='vae')
    
    
    reconstruction_loss = binary_crossentropy(input_encoder, output_vae) * (config.image_size * config.image_size)
    reconstruction_loss = K.mean(reconstruction_loss)

    kl_loss = 1 + latent_log_var - K.square(latent_mu) - K.exp(latent_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5

    vae_loss = K.mean(reconstruction_loss + kl_loss)

    vae.add_loss(vae_loss)  
    vae.add_metric(reconstruction_loss, name='reconstruction_loss')
    vae.add_metric(kl_loss, name='kl_divergence_loss')

    return vae, encoder, decoder

In [ ]:
vae, encoder, decoder = VAE()
vae.compile(optimizer=config.optimizer)
    
vae.build(input_shape=(config.image_size, config.image_size, config.channels))
encoder.summary()
decoder.summary()
vae.summary()

In [ ]:
plot_model(encoder, to_file='encoder.png', show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(decoder, to_file='decoder.png', show_shapes=True, show_layer_names=True)

In [ ]:
if os.path.isfile("best_model.h5"):
    vae.load_weights('best_model.h5')

In [ ]:
vae.fit(train_dataset,
        epochs=config.n_epochs,
        callbacks=config.callbacks,
        validation_data=valid_dataset)

In [ ]:
vae.load_weights('./best_model.h5')

## Reconstruction result

In [ ]:
test_dataset = build_dataset(valid_paths, test=True)

In [ ]:
data = list(test_dataset.take(20))

fig = plt.figure(figsize=(30, 10))
for n in range(0, 20, 2):
    image = vae.predict(data[n])
    
    plt.subplot(2, 10, n + 1)
    plt.imshow(np.squeeze(data[n]))
    plt.title('original image')
    
    plt.subplot(2, 10, n + 2)
    plt.imshow(np.squeeze(image))
    plt.title('reconstruct')
    
plt.show()

## Generation new faces

In [ ]:
n_show_new_images = 60
random_codes = np.random.normal(size=(n_show_new_images, config.latent_space_dim))
new_faces = decoder.predict(np.array(random_codes))

fig = plt.figure(figsize=(30, 15))

for i in range(n_show_new_images):
    ax = fig.add_subplot(6, 10, i+1)
    ax.imshow(new_faces[i])
    ax.axis('off')
plt.show()

## Show latent space distribution

In [ ]:
z_test = encoder.predict(test_dataset, verbose = 1)

In [ ]:
from scipy.stats import norm

z_test_samples = z_test[2]
x = np.linspace(-3, 3, 100)

fig = plt.figure(figsize=(30, 15))
fig.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(50):
    ax = fig.add_subplot(5, 10, i+1)
    ax.hist(z_test_samples[:, i], density=True, bins = 20)

    ax.text(0.5, -0.35, str(i), fontsize=10, ha='center', transform=ax.transAxes)
    ax.plot(x, norm.pdf(x))

plt.show()